> # 👨‍💻Sartourius Cell Instance Segmentation with UNet👨‍💻

                                       🤖Training Version🤖


In [ ]:
!pip install wandb
import wandb
# Wandb Login
wandb.login()
# wandb config
#WANDB_CONFIG = {'competition': 'Sartorius', '_wandb_kernel': 'neuracort', 'entity':"kohyun1207"}
run = wandb.init(project='sartorius-unet')

In [ ]:
#Libarary
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random
import skimage.morphology
import warnings
warnings.filterwarnings(action='ignore')

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F

#Initializing All seed
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
# When running on the CuDNN backend, two further options must be set
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Set a fixed value for the hash seed
os.environ['PYTHONHASHSEED'] = str(seed)
print('> SEEDING DONE')

#Groupping Annotation by 'ID'
train_info = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
df = train_info.groupby('id').agg(list).reset_index()
for col in df.columns[2:]:
    df[col] = df[col].apply(lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x))
df.head(5)

> # Utils Functions🔨

1. Decoder : function to decode to 2d array (height, width) from 1d array

2. Visulization : Check the each image druing training (input, pred, actual)

In [ ]:
#Utils
def visualization(photo, predict, actual):
    idx = np.random.randint(0, len(df))
    photo = np.transpose(photo[0].cpu().numpy(), (1,2,0))
    predict = np.transpose(predict[0].cpu().numpy(), (1,2,0))
    actual = np.transpose(actual[0].cpu().numpy(), (1,2,0))
    plt.figure(figsize = (10,10))
    plt.subplot(1,3,1)
    plt.title('Photo (X)')
    plt.imshow(photo)
    plt.tight_layout()
    plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
    plt.subplot(1,3,2)
    plt.title('Prdict(y_hat)')
    plt.imshow(predict)
    plt.tight_layout()
    plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
    plt.subplot(1,3,3)
    plt.title('Actual(yt)')
    plt.imshow(actual)
    plt.tight_layout()
    plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
    wandb.log({"chart": plt})
    #plt.show()

def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

> # IOU Score

Check IOU Score btw prediction ↔️ actual to figure out the best Thresh hold

![](https://www.pyimagesearch.com/wp-content/uploads/2016/09/iou_examples.png)

In [ ]:
#https://www.kaggle.com/theoviel/competition-metric-map-iou
def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    iou = intersection / union
    
    return iou[1:, 1:]  # exclude background

def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array [n_truths x n_preds]): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects
    false_positives = np.sum(matches, axis=0) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn

def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred) for truth, pred in zip(truths, preds)]
    
    print(ious[0].shape)

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)

#iou_map([masks] * 5, [masks] * 5, verbose=1)  # This should score 1

> # **Load Dataset**


In [ ]:
class cumstom_nomalize(object):
    def __call__(self, inputs):
        mean = 0.5 
        std = 0.5
        inputs = ((inputs * mean) / std)
        return inputs


train_transforms = T.Compose([T.Resize((224,224)),
                             T.RandomHorizontalFlip(p = 0.5),
                             T.RandomVerticalFlip(p = 0.5),
                             T.ToTensor(),
                             cumstom_nomalize()])

valid_transforms = T.Compose([T.Resize((224, 224)), T.ToTensor(), cumstom_nomalize()])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df ,transforms = None, seed = None):
        self.df = df
        self.transforms = transforms
        self.seed = seed

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = '../input/sartorius-cell-instance-segmentation/train/{}.png'.format(self.df['id'].iloc[idx])
        inputs = Image.open(path).convert('RGB')
        labels = self.df['annotation'].iloc[idx]
        mask = np.zeros((520, 704))
        for label in labels:
                mask += rle_decode(label, shape=(520, 704))
        mask = mask.clip(0, 1)
        mask = Image.fromarray(np.float32(mask))
        
        if self.transforms:
            torch.manual_seed(self.seed)
            inputs = self.transforms(inputs)
        if self.transforms:
            torch.manual_seed(self.seed)
            mask = self.transforms(mask)

        return inputs, mask

> # **UNet architecture**

In [ ]:
##Network (Unet)
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        def CBR(in_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = True):
            layers = []
            layers += [nn.Conv2d(in_channels= in_channels, out_channels =out_channels,
                                 kernel_size = kernel_size, stride = stride , padding = padding, bias = bias)]
            layers += [nn.BatchNorm2d(num_features = out_channels)]
            layers += [nn.ReLU()]
            
            return nn.Sequential(*layers)
        
        self.step1_1 = CBR(in_channels = 3, out_channels = 64)
        self.step1_2 = CBR(in_channels = 64, out_channels = 64)
        self.pool1 = nn.MaxPool2d(2)
        self.step2_1 = CBR(in_channels = 64, out_channels = 128)
        self.step2_2 = CBR(in_channels = 128, out_channels = 128)
        self.pool2 = nn.MaxPool2d(2)
        self.step3_1 = CBR(in_channels = 128, out_channels = 256)
        self.step3_2 = CBR(in_channels = 256, out_channels = 256)
        self.pool3 = nn.MaxPool2d(2)
        self.step4_1 = CBR(in_channels = 256, out_channels = 512)
        self.step4_2 = CBR(in_channels = 512, out_channels = 512)
        self.pool4 = nn.MaxPool2d(2)
        self.step5_1 = CBR(in_channels = 512, out_channels = 1024)
        
        self.rstep5_1 = CBR(in_channels = 1024, out_channels = 512)
        self.unpool4 = nn.ConvTranspose2d(in_channels = 512, out_channels = 512, kernel_size = 2, stride = 2, padding = 0, bias = True)
        self.rstep4_2 = CBR(in_channels = 512 * 2 , out_channels = 512)
        self.rstep4_1 = CBR(in_channels = 512, out_channels = 256)
        self.unpool3 = nn.ConvTranspose2d(in_channels = 256, out_channels = 256, kernel_size = 2, stride = 2, padding = 0, bias = True)
        self.rstep3_2 = CBR(in_channels = 256 * 2, out_channels = 256)
        self.rstep3_1 = CBR(in_channels = 256, out_channels = 128)
        self.unpool2 = nn.ConvTranspose2d(in_channels = 128, out_channels = 128, kernel_size = 2, stride = 2, padding = 0, bias = True)
        self.rstep2_2 = CBR(in_channels = 128 * 2, out_channels = 128)
        self.rstep2_1 = CBR(in_channels = 128, out_channels = 64)
        self.unpool1 = nn.ConvTranspose2d(in_channels = 64, out_channels = 64, kernel_size = 2, stride = 2, padding = 0, bias = True)
        self.rstep1_2 = CBR(in_channels = 64 * 2, out_channels = 64)
        self.rstep1_1 = CBR(in_channels = 64, out_channels = 64)
        
        self.fc = nn.Conv2d(in_channels = 64, out_channels = 1, kernel_size = 1, stride = 1, padding = 0, bias = True)
        
    def forward(self, x):
        step1_1 = self.step1_1(x)
        step1_2 = self.step1_2(step1_1)
        pool1 = self.pool1(step1_2)
        
        step2_1 = self.step2_1(pool1)
        step2_2 = self.step2_2(step2_1)
        pool2 = self.pool2(step2_2)
        
        step3_1 = self.step3_1(pool2)
        step3_2 = self.step3_2(step3_1)
        pool3 = self.pool3(step3_2)
        
        step4_1 = self.step4_1(pool3)
        step4_2 = self.step4_2(step4_1)
        pool4 = self.pool4(step4_2)
        
        step5_1 = self.step5_1(pool4)
        
        rstep5_1 = self.rstep5_1(step5_1)
        
        unpool4 = self.unpool4(rstep5_1)
        cat4 = torch.cat((unpool4, step4_2), dim = 1)
        rstep4_2 = self.rstep4_2(cat4)
        rstep4_1 = self.rstep4_1(rstep4_2)
        
        unpool3 = self.unpool3(rstep4_1)
        cat3 = torch.cat((step3_2, unpool3), dim = 1)
        rstep3_2 = self.rstep3_2(cat3)
        rstep3_1 = self.rstep3_1(rstep3_2)
        
        unpool2 = self.unpool2(rstep3_1)
        cat2 = torch.cat((step2_2, unpool2), dim = 1)
        rstep2_2 = self.rstep2_2(cat2)
        rstep2_1 = self.rstep2_1(rstep2_2)
        
        unpool1 = self.unpool1(rstep2_1)
        cat1 = torch.cat((step1_2, unpool1), dim = 1)
        rstep1_2 = self.rstep1_2(cat1)
        rstep1_1 = self.rstep1_1(rstep1_2)
        output = self.fc(rstep1_1)
        
        return output

> # Training

**Basic option ....**
1. lr = 0.001
2. early_stopping = 10
3. batch_size = 32
4. Loss Function = BCEWithLogitsloss
5. Optimizer = Adam

In [ ]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()

In [ ]:
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [ ]:
#Training Configure
#basic option
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#set seed
import random
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

#Defineing UNet


model = UNet()
jacard_p = 2
lr = 1e-3
setting_patience = 20
n_epoch = 250
batch_size = 16
#optimizer & loss functions
criterion = MixedLoss(10.0, 2.0)
optimizer = torch.optim.SGD(params = model.parameters(), lr = lr)
#Data Split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size = 0.33, random_state = 777, shuffle = False)
print(X_train.shape, X_test.shape)
#Data defining
train_dataset = CustomDataset(df = X_train, transforms = train_transforms, seed = 777)
valid_dataset = CustomDataset(df = X_test, transforms = valid_transforms, seed = 777) 
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
valid_dataloader = DataLoader(valid_dataset, batch_size = batch_size, shuffle = True)
train_total_batch = len(train_dataloader)
valid_total_batch = len(valid_dataloader)
trn_loss_list = []
test_loss_list = []

In [ ]:
#Training & Evaluation
torch.cuda.empty_cache()
best_loss = 100
total_patience = 0
for epoch in range(n_epoch):
    model.train()    
    model.cuda()
    optimizer.zero_grad()
    trn_avg_loss = 0
    trn_avg_iou = 0
    test_avg_loss = 0
    with tqdm(train_dataloader, unit = 'batch') as train_bar:
        for image, label in train_bar:  
            image = image.float().cuda()
            label = label.float().cuda()
            trn_output = model(image)
            iou_score = iou_map([np.transpose(trn_output[0].cpu().detach().numpy(), (1,2,0))], [np.transpose(label[0].cpu().detach().numpy(), (1,2,0))])
            loss = criterion(label,trn_output)
            loss.backward()
            optimizer.step()
            trn_avg_loss += loss / train_total_batch
            trn_avg_iou += iou_score / train_total_batch
            train_bar.set_postfix(epoch = epoch+1, loss = loss.item(), iou_score = trn_avg_iou)
    model.eval()
    with torch.no_grad():
        with tqdm(valid_dataloader, unit = 'batch') as test_bar:
            for image, label in test_bar:  
                image = image.float().cuda()
                label = label.float().cuda()
                test_output = model(image)
                loss = criterion(label, test_output)
                test_avg_loss += loss / valid_total_batch
                test_bar.set_postfix(epoch = epoch+1, loss = loss.item())
    wandb.log({'Epoch' : epoch, "Validation loss": test_avg_loss, "Train loss": trn_avg_loss})
    trn_loss_list.append(trn_avg_loss)
    test_loss_list.append(test_avg_loss)
    if total_patience == setting_patience:
        wandb.finish()
        break
    else:
        if best_loss > test_avg_loss:
            total_patience = 0
            last_loss = best_loss
            best_loss = test_avg_loss
            print('Model Improving')
            print('Epoch : {}, Loss : {:.10f} ------> {:.10f}, differ : ({:.10f}) model save.....'.format(epoch+1, last_loss, best_loss, last_loss-best_loss))
            visualization(photo = image, predict = test_output, actual = label)
            torch.save(model.state_dict(), './checkpoint.pt')
        else:
            print('early stop counter : {}/{}'.format(total_patience+1, setting_patience))
            total_patience += 1